In [4]:
#importing the libraries
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [8]:
IMAGE_SIZE = [224,224]
train_path = 'Datasets/train'
valid_path = 'Datasets/test'

In [11]:
# Importing resnet50 library and add preprocessing layer
resnet = ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet',include_top=False)
## The [3] is number of channels, in our case RGB

In [15]:
for layer in resnet.layers:
    layer.trainable = False

In [16]:
folders = glob('Datasets/train/*')

In [17]:
folders

['Datasets/train\\audi',
 'Datasets/train\\lamborghini',
 'Datasets/train\\mercedes']

In [19]:
# Flatten the resnet output
x = Flatten()(resnet.output)

In [21]:
prediction = Dense(len(folders), activation= 'softmax')(x)
#create model object
model = Model(inputs=resnet.input, outputs = prediction)

In [23]:
# Tell the model what cost and optimization method to use
model.compile(
    loss = 'categorical_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

In [25]:
# Use image data generator to import the images from dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale= 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale= 1./255)